In [13]:
import matplotlib as plt
import pandas as pd
import numpy as np
import bs4
import re
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup

In [14]:
keyWord = 'data science'
keyWord = keyWord.replace(' ', '%20')
urlorig='https://www.headhunter.com/jobs?Keywords='+keyWord+'&Radius=150&PageNumber=1&OrderBy=Relevance'
urlorig

'https://www.headhunter.com/jobs?Keywords=data%20science&Radius=150&PageNumber=1&OrderBy=Relevance'

In [123]:
my_url = urlorig
uClient = uReq(my_url)
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,'html.parser')
#     page_soup = soup(page_html,'lxml')
print(page_soup)

<!DOCTYPE html>

<html>
<head>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"13840fb14e","applicationID":"147043153","transactionName":"Jg0IFxRbWVhTEEkBBjoLCBcKG19bVBFJCAsHEAMQE1hBRw==","queueTime":0,"applicationTime":889,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHan

In [84]:

my_url1 = "https://www.headhunter.com/jobs/Hands-On-Manager---Director-of-Data-Science/J3S1KQ68RKF72TJVFRM"
uClient1 = uReq(my_url1)
page_html1 = uClient1.read()
uClient1.close()
page_soup1 = soup(page_html1,'html.parser')

print(page_soup1)



<!DOCTYPE html>

<html>
<head>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"13840fb14e","applicationID":"147043153","transactionName":"Jg0IFxRbWVhTEEkBBjoLCBcKG19bVBFJBgERAw8P","queueTime":0,"applicationTime":1218,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(3),u=e(4),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=["setPageViewName","setCustomAttribute","setErrorHandler","

In [148]:
content = page_soup1.find("div", class_="content")
print(content.text)
refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
print(refID.group()[8:])


Ref ID: 02710-9502212681Classification: Hands-On Manager / Director of Data ScienceCompensation: $100000.00 to $150000.00 yearlyThe Manager/Director, Data Science will be responsible for helping develop the companies data science strategy, building decision systems with machine learning and surfacing company-wide actionable insights to drive business growth. This position is ideal for someone who enjoys elevating the way decision are made and data is used at a fast-paced, consumer-facing company. A successful candidate will enjoy researching and developing statistical learning models, collaborating with various stakeholders to understand company needs / devise solutions and helping implement data driven strategies that impact business profitability. This is a multifaceted opportunity allowing the Manager to deliver unique value propositions to the companies base of clients. Duties/Responsibilities: •Help define, lead and own data science strategy, including tools and team •Apply artif

In [144]:
titles = []
company_names = []
location_names = []
compensation = []
job_descriptions = []
job_ids = []
skills = {}
skills['Python'] = []
skills['R '] = []
skills['Matlab'] = []
skills['Excel'] = []
skills['SAS'] = []
skills['SQL'] = []
skills['SPSS'] = []
skills['Hadoop'] = []
skills['Spark'] = []

# extract job_title
jobs = page_soup.findAll("div", class_=re.compile("searchresult clearfix"))
for job in jobs:
    try:
        titles.append(job["title"])
    except:
        titles.append("NA")

# print(titles)

# extract company_name
companies = page_soup.findAll("span", class_="resultcompany_mobile clearfix" )
for company in companies:
    try:
        company_names.append(company.text.strip())
    except:
        company_names.append("NA")
        
# print(company_names)

# extract location       
locations = page_soup.findAll("span", class_="result-right-one")
for location in locations:
    try:
        location_names.append(location.text)
    except:
        location_names.append("NA")
        
# print(location_names)

# extract compensation       
payments = page_soup.findAll("span", class_="result-right-two non-mobile")
for payment in payments:
    try:
        compensation.append(payment.text)
    except:
        compensation.append("NA")
        
# print(compensation)

links = page_soup.findAll("a", class_ = "result-left")
for link in links:
    Jb = uReq("https://www.headhunter.com" + link["href"])
    Jb_html = Jb.read()
    Jb.close()
    Jb_soup = soup(Jb_html, "html.parser")
    try:
        content = Jb_soup.find("div", class_="content")
        refID = re.search("Ref ID: [0-9]+-+[0-9]+", content.text)
        job_ids.append(refID.group()[8:])
    except:
        job_ids.append("NA")    
        
    try:
        job_descriptions.append(str(re.sub('(\n|\r)', '',content.text)))
    except:
        job_descriptions.append("NA")
        
print(job_descriptions)

# # extract job description
# job_descrs = page_soup.findAll("div", class_= "resultlower non-mobile")
# for job_descr in job_descrs:
#     try:
#         job_descriptions.append(str(re.sub('(\n|\r)', '',job_descr.p.text)))
# #         job_descriptions.append(job_descr.p.text)
#     except:
#         job_descriptions.append("NA")  

# print(job_descriptions)

['Ref ID: 02710-9502212681Classification: Hands-On Manager / Director of Data ScienceCompensation: $100000.00 to $150000.00 yearlyThe Manager/Director, Data Science will be responsible for helping develop the companies data science strategy, building decision systems with machine learning and surfacing company-wide actionable insights to drive business growth. This position is ideal for someone who enjoys elevating the way decision are made and data is used at a fast-paced, consumer-facing company. A successful candidate will enjoy researching and developing statistical learning models, collaborating with various stakeholders to understand company needs / devise solutions and helping implement data driven strategies that impact business profitability. This is a multifaceted opportunity allowing the Manager to deliver unique value propositions to the companies base of clients. Duties/Responsibilities: •Help define, lead and own data science strategy, including tools and team •Apply arti

In [145]:
# d = {'Job_ID':job_ids,'Job_Title':titles,'Company_Name':company_names,'Location':location_names,'Job_Description':job_descriptions}
d = {'Job_Title':titles,'Company_Name':company_names, 'Location':location_names, 'Compensation': compensation, 'Job_ID':job_ids,'Job_Description':job_descriptions}
df1 = pd.DataFrame(d)
df2 = pd.DataFrame(skills)
frames = [df1, df2]
results = pd.concat(frames, axis = 1)
results.head()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Python,R,Matlab,Excel,SAS,SQL,SPSS,Hadoop,Spark
0,Hands-On-Manager---Director-of-Data-Science,Robert Half Technology,NJ - Hoboken,$100k - $150k/year,02710-9502212681,Ref ID: 02710-9502212681Classification: Hands-...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Senior-Data-Scientist,Robert Half Technology,TX - Fort Worth,N/A,04030-0011057821,Ref ID: 04030-0011057821Classification: Data S...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Senior-Data-Engineer,Robert Half Technology,TX - Austin,$115k - $130k/year,04160-0010956620,Ref ID: 04160-0010956620Classification: Big Da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Sr--Data-Engineer,Robert Half Technology,FL - Riverview,$100k - $120k/year,01070-9502210262,Ref ID: 01070-9502210262Classification: Sr. Da...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Data-Integration-Delivery-Manager,"TEKsystems, Inc",MD - Baltimore,N/A,NA,TEKsystems Global Services® is part of TEKsyst...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [146]:
results.tail()

,Job_Title,Company_Name,Location,Compensation,Job_ID,Job_Description,Python,R,Matlab,Excel,SAS,SQL,SPSS,Hadoop,Spark
24,Technical-Manager,SORA Labs,MO - Forsyth,N/A,NA,SUMMARY:The technical manager’s primary functi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,SAP-PO---Seeburger-Consultant,AIC (part of ACS Group),TX - Irving,N/A,NA,SAP PO & Seeburger Consultant - 12 Months Cont...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,Sr--Test-Engineer,AIC (part of ACS Group),GA - Hapeville,N/A,NA,Qualifications:* 6 + years of experience requ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Sr-BI-Analyst-AB-Tester,AIC (part of ACS Group),TX - Irving,N/A,NA,Position: Sr BI Analyst/AB TesterLocation: Irv...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Program-Manager-or-Leader--Pharmaceutical,AIC (part of ACS Group),NJ - New Brunswick,N/A,NA,Title: Program Leader (Manager)Location: New B...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [147]:
results.to_csv('Headhunters_job_vacancies.csv')